## Capture Fataility Data from OSHA

Test getting 1 row from API line and converting it to a dataframe

In [1]:
import requests
import json
import pandas as pd

#Get a API Key From https://devtools.dol.gov/developer
apiKey = 'xxxxxx'
headers = {'Accept': 'application/json', "Authorization": "ApiKey=" + apiKey}

url = 'https://api.dol.gov/V1/Safety/Fatalities/OSHA_Fatalities/?$top=1'

req = requests.get(url,headers=headers)
osha_return = req.json()
print(osha_return)

{'d': [{'__metadata': {'uri': 'https://api.dol.gov/V1/Safety/Fatalities/OSHA_Fatalities(1L)', 'type': 'FatalitiesModel.OSHA_Fatalities'}, 'id': '1', 'dateofincident': '/Date(988848000000)/', 'companyandlocation': 'showpay LLC, Orlando, FL 32821', 'description': 'Worker was struck by a stack of industrial lighting that was being raised on a forklift.'}]}


In [2]:
osha_return['d'][0]

{'__metadata': {'uri': 'https://api.dol.gov/V1/Safety/Fatalities/OSHA_Fatalities(1L)',
  'type': 'FatalitiesModel.OSHA_Fatalities'},
 'id': '1',
 'dateofincident': '/Date(988848000000)/',
 'companyandlocation': 'showpay LLC, Orlando, FL 32821',
 'description': 'Worker was struck by a stack of industrial lighting that was being raised on a forklift.'}

## Extract the data we want from the API request

In [3]:
print(osha_return['d'][0]['id'])
print(osha_return['d'][0]['dateofincident'])
print(osha_return['d'][0]['companyandlocation'])
print(osha_return['d'][0]['description'])

1
/Date(988848000000)/
showpay LLC, Orlando, FL 32821
Worker was struck by a stack of industrial lighting that was being raised on a forklift.


## View Full API request (100 rows)
The dataset returns a "Next" url to use to get the following result set.

In [4]:
url = 'https://api.dol.gov/V1/Safety/Fatalities/OSHA_Fatalities/'

req = requests.get(url,headers=headers)
osha_return = req.json()
print(len(osha_return['d']['results']))
print(osha_return['d']['__next'])

100
https://api.dol.gov/V1/Safety/Fatalities/OSHA_Fatalities/?$skiptoken=100L


In [5]:
while len(osha_return['d']['results']) == 100 and '__next' in osha_return['d'].keys():
    next_url = osha_return['d']['__next']
    next_req = requests.get(next_url,headers=headers)
    osha_return = next_req.json()

## Testing iteration to loop through full 100 but also stop when there is not next series url without failing

In [18]:
req = requests.get(url,headers=headers)
osha_return = req.json()

while len(osha_return['d']['results']) == 100 and '__next' in osha_return['d'].keys():
    print(str(len(osha_return['d']['results'])) + "     |     " + osha_return['d']['__next'])
    next_url = osha_return['d']['__next']
    next_req = requests.get(next_url,headers=headers)
    osha_return = next_req.json()

print(str(len(osha_return['d']['results'])) + next_url)

100     |     https://api.dol.gov/V1/Safety/Fatalities/OSHA_Fatalities/?$skiptoken=100L
100     |     https://api.dol.gov/V1/Safety/Fatalities/OSHA_Fatalities/?$skiptoken=200L
100     |     https://api.dol.gov/V1/Safety/Fatalities/OSHA_Fatalities/?$skiptoken=300L
100     |     https://api.dol.gov/V1/Safety/Fatalities/OSHA_Fatalities/?$skiptoken=400L
100     |     https://api.dol.gov/V1/Safety/Fatalities/OSHA_Fatalities/?$skiptoken=500L
100     |     https://api.dol.gov/V1/Safety/Fatalities/OSHA_Fatalities/?$skiptoken=600L
100     |     https://api.dol.gov/V1/Safety/Fatalities/OSHA_Fatalities/?$skiptoken=700L
100     |     https://api.dol.gov/V1/Safety/Fatalities/OSHA_Fatalities/?$skiptoken=800L
100     |     https://api.dol.gov/V1/Safety/Fatalities/OSHA_Fatalities/?$skiptoken=900L
100     |     https://api.dol.gov/V1/Safety/Fatalities/OSHA_Fatalities/?$skiptoken=1000L
100     |     https://api.dol.gov/V1/Safety/Fatalities/OSHA_Fatalities/?$skiptoken=1100L
100     |     https://api.dol.

## Completed Script to download all the OSHA Fatality Data to a csv file.

In [43]:
import requests
import json
import pandas as pd

returned_data = []

#Get a API Key From https://devtools.dol.gov/developer
apiKey = 'a6c7457e-61de-4ba8-8154-4e6842d9b9bc'
headers = {'Accept': 'application/json', "Authorization": "ApiKey=" + apiKey}

url = 'https://api.dol.gov/V1/Safety/Fatalities/OSHA_Fatalities/'

req = requests.get(url,headers=headers)
osha_return = req.json()

for record in osha_return['d']['results']:
    returned_data.append(record)
    
    
while len(osha_return['d']['results']) == 100 and '__next' in osha_return['d'].keys():
    next_url = osha_return['d']['__next']
    next_req = requests.get(next_url,headers=headers)
    osha_return = next_req.json()
    for records in osha_return['d']['results']:
        returned_data.append(records)
    
    
last_url = next_url
last_req = requests.get(last_url,headers=headers)
last_return = last_req.json()
for record_end in osha_return['d']['results']:
    returned_data.append(record_end)

df = pd.DataFrame.from_dict(returned_data)
df
#df.to_csv('output.csv', sep=',')

,__metadata,id,dateofincident,companyandlocation,description
0,{'uri': 'https://api.dol.gov/V1/Safety/Fatalit...,1,/Date(988848000000)/,"showpay LLC, Orlando, FL 32821",Worker was struck by a stack of industrial lig...
1,{'uri': 'https://api.dol.gov/V1/Safety/Fatalit...,2,/Date(989712000000)/,"AEG Live Productions, LLC Kansas City, MO 64108",Worker putting together a concert stage died w...
2,{'uri': 'https://api.dol.gov/V1/Safety/Fatalit...,3,/Date(990403200000)/,"Dynamic Auto Brokers, Inc., Sussex, NJ 07461",Worker removing a downed tree was electrocuted...
3,{'uri': 'https://api.dol.gov/V1/Safety/Fatalit...,4,/Date(991612800000)/,"Prevail Energy, Incorporated, Corpus Christi, ...",A worker was killed when a tractor being loade...
4,{'uri': 'https://api.dol.gov/V1/Safety/Fatalit...,5,/Date(991872000000)/,"Interstate Waste Services, Inc., Chester, NY 1...",Waste disposal worker crossing street to colle...
...,...,...,...,...,...
2843,{'uri': 'https://api.dol.gov/V1/Safety/Fatalit...,2820,/Date(1330992000000)/,"Epstein Construction Inc., Chicago, IL 60661",Construction manager driving his truck on work...
2844,{'uri': 'https://api.dol.gov/V1/Safety/Fatalit...,2821,/Date(1330992000000)/,"National Steel City LLC, Calvert, AL 36513",Worker installing duct work in building under ...
2845,{'uri': 'https://api.dol.gov/V1/Safety/Fatalit...,2822,/Date(1330992000000)/,"Berlanga General Painting, Eagle Pass, TX 78852",Worker using an aerial lift to paint air ducts...
2846,{'uri': 'https://api.dol.gov/V1/Safety/Fatalit...,2823,/Date(1331078400000)/,"sieveking Inc., Hazelwood, MO 63042",Worker using a portable ladder to hook a hose ...
